![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# RAG from scratch with the Redis Vector Library


In this recipe we will cover the basic of the Redis Vector Library and build a basic RAG app from scratch.

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/RAG/01_redisvl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Environment Setup

### Pull Github Materials
Because you are likely running this notebook in **Google Colab**, we need to first
pull the necessary dataset and materials directly from GitHub.

**If you are running this notebook locally**, FYI you may not need to perform this
step at all.

In [1]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/RAG/resources .
!rm -rf temp_repo

Cloning into 'temp_repo'...
remote: Enumerating objects: 384, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 384 (delta 135), reused 153 (delta 74), pack-reused 137 (from 1)
Receiving objects: 100% (384/384), 64.50 MiB | 8.97 MiB/s, done.
Resolving deltas: 100% (159/159), done.


### Install Python Dependencies

In [2]:
%pip install -q redis "redisvl>=0.4.1" langchain-community pypdf sentence-transformers langchain openai pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00


### Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.**

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [3]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [2]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

## Simplified Vector Search with RedisVL

### Dataset Preparation (PDF Documents)

To best demonstrate Redis as a vector database layer, we will load a single
financial (10k filings) doc and preprocess it using some helpers from LangChain:

- `PyPDFLoader` is not the only document loader type that LangChain provides. Docs: https://python.langchain.com/docs/integrations/document_loaders/pypdfloader/
- `RecursiveCharacterTextSplitter` is what we use to create smaller chunks of text from the doc. Docs: https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

# Load list of pdfs from a folder
data_path = "resources/"
docs = [os.path.join(data_path, file) for file in os.listdir(data_path)]

print("Listing available documents ...", docs)

Listing available documents ... ['resources/nke-10k-2023.pdf', 'resources/amzn-10k-2023.pdf', 'resources/jnj-10k-2023.pdf', 'resources/aapl-10k-2023.pdf', 'resources/testset_15.csv', 'resources/retrieval_basic_rag_test.csv', 'resources/2022-chevy-colorado-ebrochure.pdf', 'resources/nvd-10k-2023.pdf', 'resources/testset.csv', 'resources/msft-10k-2023.pdf', 'resources/propositions.json', 'resources/generation_basic_rag_test.csv']


In [6]:
# pick out the Nike doc for this exercise
doc = [doc for doc in docs if "nke" in doc][0]

# set up the file loader/extractor and text splitter to create chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500, chunk_overlap=0
)
loader = PyPDFLoader(doc, headers = None)

# extract, load, and make chunks
chunks = loader.load_and_split(text_splitter)

print("Done preprocessing. Created", len(chunks), "chunks of the original pdf", doc)

Done preprocessing. Created 211 chunks of the original pdf resources/nke-10k-2023.pdf


### Text embedding generation with RedisVL
RedisVL has built-in extensions and utilities to aid the GenAI development process. In the following snipit we utilize the HFTextVectorizer redisvl in tandem with the **all-MiniLM-L6-v2** class to generate vector embeddings for the chunks created above. These embeddings capture the "meaning" of the text so that we can retrieve the relevant chunks later when a user's query is semantically related.

In [7]:
import warnings
import pandas as pd
from tqdm.auto import tqdm
from redisvl.utils.vectorize import HFTextVectorizer

warnings.filterwarnings("ignore")

hf = HFTextVectorizer("sentence-transformers/all-MiniLM-L6-v2")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Embed each chunk content
embeddings = hf.embed_many([chunk.page_content for chunk in chunks])

# Check to make sure we've created enough embeddings, 1 per document chunk
len(embeddings) == len(chunks)

True

### Define a schema and create an index

Below we connect to Redis and create an index that contains a text field, tag field, and vector field.

In [5]:
from redisvl.index import SearchIndex


index_name = "redisvl"


schema = {
  "index": {
    "name": index_name,
    "prefix": "chunk"
  },
  "fields": [
    {
        "name": "chunk_id",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "content",
        "type": "text"
    },
    {
        "name": "text_embedding",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "hnsw",
            "datatype": "float32"
        }
    }
  ]
}

In [ ]:
# create an index from schema and the client
index = SearchIndex.from_dict(schema, redis_url=REDIS_URL)
index.create(overwrite=True, drop=True)

In [8]:
# use the RedisVL CLI tool to list all indices
!rvl index listall

14:47:36 [RedisVL] INFO   Indices:
14:47:36 [RedisVL] INFO   1. redisvl
14:47:36 [RedisVL] INFO   2. movies


In [9]:
# get info about the index
!rvl index info -i redisvl



Index Information:
╭──────────────┬────────────────┬────────────┬─────────────────┬────────────╮
│ Index Name   │ Storage Type   │ Prefixes   │ Index Options   │   Indexing │
├──────────────┼────────────────┼────────────┼─────────────────┼────────────┤
│ redisvl      │ HASH           │ ['chunk']  │ []              │          0 │
╰──────────────┴────────────────┴────────────┴─────────────────┴────────────╯
Index Fields:
╭────────────────┬────────────────┬────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬─────────────────┬────────────────┬────────────────┬────────────────┬─────────────────┬────────────────╮
│ Name           │ Attribute      │ Type   │ Field Option   │ Option Value   │ Field Option   │ Option Value   │ Field Option   │   Option Value │ Field Option    │ Option Value   │ Field Option   │   Option Value │ Field Option    │   Option Value │
├────────────────┼────────────────┼────────┼────────────────┼────────────

### Process and load dataset
Below we use the RedisVL index to simply load the list of document chunks to Redis db.

In [10]:
# load expects an iterable of dictionaries
from redisvl.redis.utils import array_to_buffer

data = [
    {
        'chunk_id': i,
        'content': chunk.page_content,
        # For HASH -- must convert embeddings to bytes
        'text_embedding': array_to_buffer(embeddings[i], dtype='float32')
    } for i, chunk in enumerate(chunks)
]

# RedisVL handles batching automatically
keys = index.load(data, id_field="chunk_id")

### Query the database
Now we can use the RedisVL index to perform similarity search operations with Redis

In [11]:
from redisvl.query import VectorQuery

query = "Nike profit margins and company performance"

query_embedding = hf.embed(query)

vector_query = VectorQuery(
    vector=query_embedding,
    vector_field_name="text_embedding",
    num_results=3,
    return_fields=["chunk_id", "content"],
    return_score=True
)

# show the raw redis query
str(vector_query)

'*=>[KNN 3 @text_embedding $vector AS vector_distance] RETURN 3 chunk_id content vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 3'

In [12]:
# execute the query with RedisVL
result=index.query(vector_query)

# view the results
pd.DataFrame(result)

,id,vector_distance,chunk_id,content
0,chunk:87,0.334264576435,87,"Asia Pacific & Latin America 1,932 1,896 2 % 1..."
1,chunk:79,0.350993335247,79,Table of Contents\nCONSOLIDA TED OPERA TING RE...
2,chunk:86,0.371814072132,86,Table of Contents\nOPERA TING SEGMENTS\nAs dis...


In [13]:
# paginate through results
for result in index.paginate(vector_query, page_size=1):
    print(result[0]["chunk_id"], result[0]["vector_distance"], flush=True)

87 0.334264576435
79 0.350993335247
86 0.371814072132


### Sort by alternative fields

In [14]:
# Sort by chunk_id field after vector search limits to topK
vector_query = VectorQuery(
    vector=query_embedding,
    vector_field_name="text_embedding",
    num_results=4,
    return_fields=["chunk_id"],
    return_score=True
)

# Decompose vector_query into the core query and the params
query = vector_query.query
params = vector_query.params

# Pass query and params direct to index.search()
result = index.search(
    query.sort_by("chunk_id", asc=True),
    params
)

pd.DataFrame([doc.__dict__ for doc in result.docs])


,id,payload,vector_distance,chunk_id
0,chunk:79,None,0.350993335247,79
1,chunk:82,None,0.378765702248,82
2,chunk:86,None,0.371814072132,86
3,chunk:87,None,0.334264576435,87


### Add filters to vector queries

In [15]:
from redisvl.query.filter import Text

vector_query = VectorQuery(
    vector=query_embedding,
    vector_field_name="text_embedding",
    num_results=4,
    return_fields=["content"],
    return_score=True
)

# Set a text filter
text_filter = Text("content") % "profit"

vector_query.set_filter(text_filter)

result=index.query(vector_query)
pd.DataFrame(result)

,id,vector_distance,content
0,chunk:82,0.378765702248,Table of Contents\nGROSS MARGIN\nFISCAL 2023 C...
1,chunk:128,0.418757259846,"Table of Contents\nNIKE, INC.\nCONSOLIDATED ST..."
2,chunk:72,0.466709017754,Table of Contents\nITEM 7. MANAGEM ENT'S DISCU...
3,chunk:62,0.493393957615,"existing businesses, such as our NIKE Direct o..."


### Range queries in RedisVL

In [16]:
from redisvl.query import RangeQuery

range_query = RangeQuery(
    vector=query_embedding,
    vector_field_name="text_embedding",
    num_results=4,
    return_fields=["content"],
    return_score=True,
    distance_threshold=0.8  # find all items with a semantic distance of less than 0.8
)

In [17]:
result=index.query(range_query)
pd.DataFrame(result)

,id,vector_distance,content
0,chunk:87,0.334264576435,"Asia Pacific & Latin America 1,932 1,896 2 % 1..."
1,chunk:79,0.350993335247,Table of Contents\nCONSOLIDA TED OPERA TING RE...
2,chunk:86,0.371814072132,Table of Contents\nOPERA TING SEGMENTS\nAs dis...
3,chunk:82,0.378765702248,Table of Contents\nGROSS MARGIN\nFISCAL 2023 C...


In [18]:
# Add filter to range query
range_query.set_filter(text_filter)

index.query(range_query)
pd.DataFrame(result)

,id,vector_distance,content
0,chunk:87,0.334264576435,"Asia Pacific & Latin America 1,932 1,896 2 % 1..."
1,chunk:79,0.350993335247,Table of Contents\nCONSOLIDA TED OPERA TING RE...
2,chunk:86,0.371814072132,Table of Contents\nOPERA TING SEGMENTS\nAs dis...
3,chunk:82,0.378765702248,Table of Contents\nGROSS MARGIN\nFISCAL 2023 C...


## Building a basic RAG Pipeline from Scratch
We're going to build a basic RAG pipeline from scratch incorporating the following components:

- Standard semantic search
- Integration with OpenAI for LLM
- Chat completion

### Setup RedisVL AsyncSearchIndex

In [ ]:
from redisvl.index import AsyncSearchIndex

async_index = AsyncSearchIndex.from_dict(schema, redis_url=REDIS_URL)

### Setup OpenAI API

In [20]:
import openai
import os
import getpass


CHAT_MODEL = "gpt-3.5-turbo-0125"

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY :")


### Baseline Retrieval Augmented Generation
The code below answers a user's questions following this basic flow:

1. Generate a query_vector from the user's chat question to have an apples to apples comparison against the vector database.
2. Retrieve the most semantically relevant chunks to the user's query from the database.
3. Pass the user query and retrieved context to the `promptify` function to generate the final prompt to be sent to the LLM along with the system prompt and necessary hyperparameters.
4. Return the LLMs response to the user.

In [21]:

async def answer_question(index: AsyncSearchIndex, query: str):
    """Answer the user's question"""

    SYSTEM_PROMPT = """You are a helpful financial analyst assistant that has access
    to public financial 10k documents in order to answer users questions about company
    performance, ethics, characteristics, and core information.
    """

    query_vector = hf.embed(query)
    # Fetch context from Redis using vector search
    context = await retrieve_context(index, query_vector)
    # Generate contextualized prompt and feed to OpenAI
    response = await openai.AsyncClient().chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": promptify(query, context)}
        ],
        temperature=0.1,
        seed=42
    )
    # Response provided by LLM
    return response.choices[0].message.content


async def retrieve_context(async_index: AsyncSearchIndex, query_vector) -> str:
    """Fetch the relevant context from Redis using vector search"""
    results = await async_index.query(
        VectorQuery(
            vector=query_vector,
            vector_field_name="text_embedding",
            return_fields=["content"],
            num_results=3
        )
    )
    content = "\n".join([result["content"] for result in results])
    return content


def promptify(query: str, context: str) -> str:
    return f'''Use the provided context below derived from public financial
    documents to answer the user's question. If you can't answer the user's
    question, based on the context; do not guess. If there is no context at all,
    respond with "I don't know".

    User question:

    {query}

    Helpful context:

    {context}

    Answer:
    '''

### Let's test it out...

In [22]:
# Generate a list of questions
questions = [
    "What is the trend in the company's revenue and profit over the past few years?",
    "What are the company's primary revenue sources?",
    "How much debt does the company have, and what are its capital expenditure plans?",
    "What does the company say about its environmental, social, and governance (ESG) practices?",
    "What is the company's strategy for growth?"
]

In [23]:
import asyncio

results = await asyncio.gather(*[
    answer_question(async_index, question) for question in questions
])

### Let's view the results

In [24]:
for i, r in enumerate(results):
    print(f"Question: {questions[i]}")
    print(f"Answer: \n {r}", "\n-----------\n")

Question: What is the trend in the company's revenue and profit over the past few years?
Answer: 
 The trend in the company's revenue and profit over the past few years shows a consistent increase. 

- Total revenues for NIKE, Inc. have increased from $44,538 million in fiscal year 2021 to $46,710 million in fiscal year 2022, and further to $51,217 million in fiscal year 2023.
- Total Earnings Before Interest and Taxes (EBIT) have also shown growth, with EBIT increasing in most segments from fiscal year 2022 to fiscal year 2023.

Overall, based on the provided financial data, the company has experienced a positive trend in both revenue and profit over the past few years. 
-----------

Question: What are the company's primary revenue sources?
Answer: 
 The company's primary revenue sources are as follows:

1. Footwear
2. Apparel
3. Equipment

These revenue sources are further broken down by geographical segments and distribution channels. 
-----------

Question: How much debt does the c

# You now have a working RAG pipeline!

As you can see, it is easy to get started with RAG and we were able to get decent chat results from this simple setup. To go beyond the basic example though see the [advanced_rag](./04_advanced_redisvl.ipynb) notebook.

This notebook covers:

- **Improving accuracy** with dense content representations and query rewriting/expansion
- **Improving performance and optimizing cost** with semantic caching
- **Improving personalization** with chat session memory.


## Cleanup

Clean up the database.

In [25]:
await async_index.client.flushall()

True